In [1]:
!mkdir -p dataset_source
!>dataset_source/0.png
!>dataset_source/1.png
!>dataset_source/2.png
!>dataset_source/3.png

In [46]:
class SampleParser():
    def __init__(self):
        pass
    
    def run(self):
        import pandas as pd
        import os
        df = pd.DataFrame(data=[[os.getcwd() + '/dataset_source/0.png', 2, 3, 4], 
                                [os.getcwd() + '/dataset_source/1.png', 12, 13, 14], 
                                [os.getcwd() + '/dataset_source/2.png', 22, 23, 24], 
                                [os.getcwd() + '/dataset_source/3.png', 32, 33, 34]],
                          columns=['path_to_rgb', 'x', 'y', 'z'])
        return df

In [75]:
import os
import json
import tqdm
import itertools
import collections
import numpy as np
import pandas as pd
import math
import pyquaternion

from linalg.linalg_utils import (form_se3,
                                 convert_global_se3_matrices_to_relative,
                                 convert_relative_se3_matrices_to_euler)


class BaseParser:
    def __init__(self,
                 sequence_directory,
                 global_csv_filename):
        self.sequence_directory = sequence_directory
        self.global_csv_filename = global_csv_filename
        self.global_csv_path = os.path.join(self.sequence_directory, self.global_csv_filename)
    
    def _load_data(self):
        pass


    def _calculate_global_pose_matrices(self):
        pass

    def _make_absolute_filepath(self):
        self.global_dataframe.path_to_rgb = self.global_dataframe.path_to_rgb.apply(
            lambda filename: os.path.join(self.directory, filename))
        self.global_dataframe.path_to_depth = self.global_dataframe.path_to_depth.apply(
            lambda filename: os.path.join(self.directory, filename))
    
    def _create_global_dataframe(self):
        pass


    def parse(self):
        self._load_data()
        self._create_global_dataframe()
        self._make_absolute_filepath()
        self.global_dataframe.to_csv(self.global_csv_path, index=False)
        print('Parse ok...')

In [81]:
class TUMParser(BaseParser):
    def __init__(self,
                 sequence_directory,
                 directory,
                 global_csv_filename='global.csv'):
        super(TUMParser, self).__init__(sequence_directory, 
                                        global_csv_filename
                                        )
        self.directory = directory
        self.gt_txt_path = os.path.join(self.directory, 'groundtruth.txt')
        self.depth_txt_path = os.path.join(self.directory, 'depth.txt')
        self.rgb_txt_path = os.path.join(self.directory, 'rgb.txt')

    @staticmethod
    def associate_timestamps(timestamps, other_timestamps, max_difference=0.02):
        timestamps = list(timestamps)
        other_timestamps = list(other_timestamps)
        potential_matches = [(timestamp, other_timestamp)
                             for timestamp in timestamps
                             for other_timestamp in other_timestamps
                             if abs(timestamp - other_timestamp) < max_difference]
        potential_matches.sort(key=lambda x: abs(x[0] - x[1]))
        
        matches = []
        for timestamp, other_timestamp in potential_matches:
            if timestamp in timestamps and other_timestamp in other_timestamps:
                timestamps.remove(timestamp)
                other_timestamps.remove(other_timestamp)
                matches.append((timestamp, other_timestamp))

        matches.sort()
        return list(zip(*matches))
    
    @staticmethod
    def associate_dataframes(dataframes, timestamp_cols):
        dataframe = dataframes[0]
        timestamp_col = timestamp_cols[0]
        for other_dataframe, other_timestamp_col in zip(dataframes[1:], timestamp_cols[1:]):
            timestamps, other_timestamps = \
                TUMParser.associate_timestamps(dataframe[timestamp_col].values, other_dataframe[other_timestamp_col].values)
            dataframe = dataframe[dataframe[timestamp_col].isin(timestamps)]
            dataframe.index = np.arange(len(dataframe))
            other_dataframe = other_dataframe[other_dataframe[other_timestamp_col].isin(other_timestamps)]
            other_dataframe.index = timestamps
            
            assert len(dataframe) == len(other_dataframe)
            dataframe = dataframe.join(other_dataframe, on=timestamp_col)
        return dataframe

    def _load_txt(self, txt_path, columns):
        dataframe = pd.read_csv(txt_path, skiprows=3, sep=' ', index_col=False, names=columns)
        dataframe.columns = columns
        timestamp_col = columns[0]
        dataframe[timestamp_col] = dataframe[timestamp_col].apply(float)
        return dataframe

    def _load_data(self):
        gt_dataframe = self._load_txt(self.gt_txt_path, columns=['timestamp_gt', 'x', 'y', 'z', 'qx', 'qy', 'qz', 'qw'])
        depth_dataframe = self._load_txt(self.depth_txt_path, columns=['timestamp_depth', 'path_to_depth'])
        rgb_dataframe = self._load_txt(self.rgb_txt_path, columns=['timestamp_rgb', 'path_to_rgb'])
        self.dataframes = [depth_dataframe, rgb_dataframe, gt_dataframe]
        self.timestamp_cols = ['timestamp_depth', 'timestamp_rgb', 'timestamp_gt']

    def _create_global_dataframe(self):
        self.global_dataframe = self.associate_dataframes(self.dataframes, self.timestamp_cols)

    def __repr__(self):
        return 'TUMParser(dir={}, txt_path={}, global_csv_filename={})'.format(
            self.sequence_directory, self.gt_txt_path, self.global_csv_filename
            )

In [87]:
sequence_directory = '/Vol0/user/f.konokhov/newodomety/odometry/notebooks/tudum'
directory = '/Vol1/dbstore/datasets/tum_rgbd_flow/data/rgbd_dataset_freiburg2_coke'

In [88]:
tum = TUMParser(sequence_directory, directory=directory)

In [89]:
tum.parse()

Parse ok...


In [104]:
class RetailBotParser(TUMParser):
    
    def __init__(self,
                 sequence_directory,
                 directory,
                 global_csv_filename='global.csv'):
        super(RetailBotParser, self).__init__(sequence_directory, 
                                        global_csv_filename
                                        )
        self.directory = directory
        self.gt_txt_path = os.path.join(self.directory, 'pose.txt')
        self.depth_txt_path = os.path.join(self.directory, 'depth.txt')
        self.rgb_txt_path = os.path.join(self.directory, 'rgb.txt')
    
    def _load_txt(self, txt_path, columns):
        dataframe = pd.read_csv(txt_path, skiprows=0, sep=' ', index_col=False, names=columns)
        dataframe.columns = columns
        timestamp_col = columns[0]
        dataframe[timestamp_col] = dataframe[timestamp_col].apply(float)
        return dataframe
    
    def _load_pic(self, txt_path, columns):
        dataframe = pd.read_csv(txt_path, skiprows=0, sep=' ', index_col=False, names=columns)
        dataframe.columns = columns
        timestamp_col = columns[0]
        pic = columns[1]
        dataframe[timestamp_col] = dataframe[timestamp_col].apply(float)
        dataframe[pic] = dataframe[pic].apply(lambda x : directory + x[1:])
        return dataframe
    
    def _load_data(self):
        gt_dataframe = self._load_txt(self.gt_txt_path, columns=['timestamp_gt', 'x', 'y', 'z', 'qx', 'qy', 'qz', 'qw'])
        depth_dataframe = self._load_pic(self.depth_txt_path, columns=['timestamp_depth', 'path_to_depth'])
        rgb_dataframe = self._load_pic(self.rgb_txt_path, columns=['timestamp_rgb', 'path_to_rgb'])
        self.dataframes = [depth_dataframe, rgb_dataframe, gt_dataframe]
        self.timestamp_cols = ['timestamp_depth', 'timestamp_rgb', 'timestamp_gt']

In [105]:
sequence_directory = '/Vol0/user/f.konokhov/newodomety/odometry/notebooks/meetingroom_04_rgbd_ir_imu_pose_KITTI_finetuned_tum2'
directory = '/dbstore/datasets/retail_bot/meetingroom_04_rgbd_ir_imu_pose'

In [106]:
retail = RetailBotParser(sequence_directory, directory=directory)

In [107]:
retail.parse()

Parse ok...


In [118]:
class DISCOMANParser(BaseParser):
    def __init__(self,
                 sequence_directory,
                 json_path,
                 global_csv_filename='global.csv'):
        super(DISCOMANParser, self).__init__(sequence_directory,
                                             global_csv_filename,)
        self.directory = os.path.dirname(json_path)
        self.image_directory = os.path.dirname(json_path)
        self.depth_directory = os.path.dirname(json_path)
        self.json_path = json_path

    def _load_data(self):
        with open(self.json_path) as read_file:
            data = json.load(read_file)
        self.trajectory = data['trajectory']['frames']

    @staticmethod    
    def get_path_to_rgb(item):
        return '{}_raycast.jpg'.format(item['id'])

    @staticmethod
    def get_path_to_depth(item):
        return '{}_depth.png'.format(item['id'])

    @staticmethod
    def get_timestamp(item):
        return item['id']

    @staticmethod
    def get_global_quaternion(item):
        return item['state']['global']['orientation']

    @staticmethod
    def get_global_translation(item):
        return item['state']['global']['position']

    @staticmethod
    def get_global_pose_matrix(item):
        return pyquaternion.Quaternion(item['state']['global']['orientation']).rotation_matrix

    def _create_global_dataframe(self):
        trajectory_parsed = []

        for point in self.trajectory[::5]:
            parsed_point = {}
            parsed_point['timestamp'] = self.get_timestamp(point)
            parsed_point['path_to_rgb'] = self.get_path_to_rgb(point)
            parsed_point['path_to_depth'] = self.get_path_to_depth(point)
            parsed_point.update(dict(zip(['qw', 'qx', 'qy', 'qz'], self.get_global_quaternion(point))))
            parsed_point.update(dict(zip(['x', 'y', 'z'], self.get_global_translation(point))))
            trajectory_parsed.append(parsed_point)

        self.global_dataframe = pd.DataFrame.from_dict(trajectory_parsed)

    def _calculate_global_pose_matrices(self):
        self.global_pose_matrices = np.array([self.get_global_pose_matrix(item) for item in self.trajectory])

    def __repr__(self):
        return 'JSONParser(dir={}, json_path={}, global_csv_filename={})'.format(
            self.sequence_directory, self.json_path, self.global_csv_filename)

In [119]:
sequence_directory = '/Vol0/user/f.konokhov/newodomety/odometry/notebooks/disco'
json_path = '/dbstore/datasets/renderbox/iros2019/dset/output/deprecated/000001/0_traj.json'

In [120]:
discoman = DISCOMANParser(sequence_directory, json_path=json_path)

In [121]:
discoman.parse()

Parse ok...


In [ ]:
class OldDISCOMANParser(DISCOMANParser):
    def __init__(self,
                 sequence_directory,
                 json_path,
                 global_csv_filename='global.csv'):
        super(OldDISCOMANParser, self).__init__(sequence_directory,
                                                json_path,
                                                global_csv_filename,
                                                relative_csv_filename,
                                                stride)

    def _load_data(self):
        with open(self.json_path) as read_file:
            data = json.load(read_file)
            self.trajectory = data['data']

    @staticmethod
    def get_path_to_rgb(item):
        return '{}_raycast.jpg'.format(str(item['time']).zfill(6))

    @staticmethod
    def get_path_to_depth(item):
        return '{}_depth.png'.format(str(item['time']).zfill(6))

    @staticmethod
    def get_timestamp(item):
        return item['time']

    @staticmethod
    def get_global_quaternion(item):
        return item['info']['agent_state']['orientation']

    @staticmethod
    def get_global_translation(item):
        return item['info']['agent_state']['position']

    @staticmethod
    def get_global_pose_matrix(item):
        return pyquaternion.Quaternion(item['info']['agent_state']['orientation']).rotation_matrix
    
    def _create_global_dataframe(self):
        trajectory_parsed = []

        for point in self.trajectory:
            parsed_point = {}
            parsed_point['timestamp'] = self.get_timestamp(point)
            parsed_point['path_to_rgb'] = self.get_path_to_rgb(point)
            parsed_point['path_to_depth'] = self.get_path_to_depth(point)
            parsed_point.update(dict(zip(['qw', 'qx', 'qy', 'qz'], self.get_global_quaternion(point))))
            parsed_point.update(dict(zip(['x', 'y', 'z'], self.get_global_translation(point))))
            trajectory_parsed.append(parsed_point)

        self.global_dataframe = pd.DataFrame.from_dict(trajectory_parsed)

In [ ]:
class KITTIParser(BaseParser):
    
    

In [2]:
class SampleEstimator():
    def __init__(self):
        pass
    
    def run(self, root, index, row):
        result = row._asdict()
        # save file <root + optical_flow_stride1 + index.npy>
        result['path_to_optical_flow_stride1'] = '{}.npy'.format(index)
        return result

In [8]:
import sys
sys.path.append('/Vol0/user/f.konokhov/newodomety/odometry/prepare_dataset/')
sys.path.append('/Vol0/user/f.konokhov/newodomety')

In [15]:
a=1

In [16]:
SampleParser.run(a)

,path_to_rgb,x,y,z
0,/Vol0/user/f.konokhov/newodomety/odometry/prep...,2,3,4
1,/Vol0/user/f.konokhov/newodomety/odometry/prep...,12,13,14
2,/Vol0/user/f.konokhov/newodomety/odometry/prep...,22,23,24
3,/Vol0/user/f.konokhov/newodomety/odometry/prep...,32,33,34


In [9]:
from odometry.prepare_dataset.prepare_trajectory import prepare_trajectory

In [5]:
prepare_trajectory('dataset_tmp', parser=SampleParser(), 
                   single_frame_estimators=[SampleEstimator()],
                   stride=1)

,path_to_rgb,x,y,z,path_to_optical_flow_stride1,path_to_rgb_next,x_next,y_next,z_next,path_to_optical_flow_stride1_next
0,dataset_tmp/rgb/0.png,2,3,4,0.npy,dataset_tmp/rgb/1.png,12,13,14,1.npy
1,dataset_tmp/rgb/1.png,12,13,14,1.npy,dataset_tmp/rgb/2.png,22,23,24,2.npy
2,dataset_tmp/rgb/2.png,22,23,24,2.npy,dataset_tmp/rgb/3.png,32,33,34,3.npy
